In [1]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [2]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [5]:
design.delete_all_components()

In [17]:
design.get_chip_z('Q_chip')

'6um'

In [3]:
design.chips['C_chip'].size['size_x']='6mm'
design.chips['C_chip'].size['size_y']='6mm'
design.chips['Q_chip'].size['size_x']='4mm'
design.chips['Q_chip'].size['size_y']='4mm'
design.chips['Q_chip'].size['center_z']='8um'

# Resonator length

In [4]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [5]:
resonator_length_4=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,4)
    a=a.strip('mm')
    a=float(a)-0.35+0.51-0.1+0.4-0.2-0.1
    
    resonator_length_4.append(a)
print(resonator_length_4)


[4.386627341935143, 4.2723941699207115, 4.164173270104566]


# Flipchip

In [6]:
cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))

# q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
#                                             pos_y='-2000um',
#                                             chip='Q_chip',
#                                             connection_pads=cp0))]
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-1300um',
                                            pos_y='-470um',
                                            chip='Q_chip',
                                            pad_width='355um',
                                            pad_height='65um',
                                            pad_gap='50um',
                                            pocket_width='650um',
                                            pocket_height='650um'))]

qubits=q0

q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = StraightUpCoupler(design, "c0", options=dict(pos_x=f"{q0x-100}um",
                                              pos_y=f"{q0y+60}um",
                                              pad_width="150um",
                                              pad_height="65um",
                                              cpw_stub_height="200um",
                                              chip="C_chip"))

design.rebuild()
gui.rebuild()
gui.autoscale()

In [7]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-1700um',
                                         pos_y = '1300um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '1700um',
                                         pos_y = '1300um',
                                         orientation='180',
                                         lead_length = '30um'))

In [8]:


stg=ShortToGround(design, 'stg', options=Dict(chip="C_chip",
                                                pos_x='-0.585mm',
                                                pos_y='0mm',
                                                orientation='270'))

In [9]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [10]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

In [11]:
jogs1=OrderedDict()
jogs1[0]=['R','350um']
jogs1[1]=['R', '400um']
jogs1[2]=['L','465um']


In [12]:
#3.91mm
rr0=connect('rr0', 'c0', 'in','stg', 'short', 
            resonator_length_4[1],jogs1,jogs, 
            '400um','0um', '1460um', '100um')

# EPR Analysis

In [13]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [14]:
hfss.start()

INFO 06:20PM [connect_project]: Connecting to Ansys Desktop API...
INFO 06:20PM [load_ansys_project]: 	Opened Ansys App
INFO 06:20PM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 06:20PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   flipchip_distance
INFO 06:20PM [connect_design]: 	Opened active design
	Design:    Design_q3d [Solution type: Q3D]
INFO 06:20PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 06:20PM [connect]: 	Connected to project "flipchip_distance" and design "Design_q3d" 😀 



True

In [15]:
hfss.activate_ansys_design("flip8um_V5", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','stg','rr0','fl0'],[],
                    box_plus_buffer=False)

06:20PM 31s WARNING [activate_ansys_design]: The design_name=flip8um_V5 was not in active project.  Designs in active project are: 
['flip6um_V5', 'flip7um_V5-1', 'Design_q3d'].  A new design will be added to the project.  
INFO 06:20PM [connect_design]: 	Opened active design
	Design:    flip8um_V5 [Solution type: Eigenmode]
WARNING 06:20PM [connect_setup]: 	No design setup detected.
WARNING 06:20PM [connect_setup]: 	Creating eigenmode default setup.
INFO 06:20PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [16]:
hfss.close()

True

# LOM Analysis

In [17]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")


In [18]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.64',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.64',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [20]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 06:28PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpjwkotzjx.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5640000000, Maxwell, 1, False
INFO 06:28PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpbvo90sk_.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5640000000, Maxwell, 1, False
INFO 06:28PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpnoaklgsv.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5640000000, Maxwell, 2, False
INFO 06:28PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpgjdcr1mp.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5640000000, Maxwell, 3, False
INFO 06:28PM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpdbekkxvt.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5640000000, Maxwell, 4, False
INFO 06:28

,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground,284.660,-78.47100,-65.855,-78.16500
pad_bot_Q0,-78.471,93.49100,-14.062,-0.13402
pad_top_Q0,-65.855,-14.06200,93.451,-12.64200
quad_coupler_c0,-78.165,-0.13402,-12.642,91.12800


In [22]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 6.96
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 6.078363 [GHz]
EC 350.038091 [MHz]
EJ 14.854145 [GHz]
alpha -408.281693 [MHz]
dispersion 53.306802 [KHz]
Lq 10.995564 [nH]
Cq 55.337486 [fF]
T1 17.312245 [us]

**Coupling Properties**

tCqbus1 -6.258382 [fF]
gbus1_in_MHz -108.673784 [MHz]
χ_bus1 -8.538401 [MHz]
1/T1bus1 9193.200638 [Hz]
T1bus1 17.312245 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,6.275934,374.88373,14.854145,-440.388251,96.801139,[-66.73484385653578],[-5.122704480925721],5.122704,66.734844
2,6.093459,351.901831,14.854145,-410.675751,55.870662,[-129.10156314594278],[-12.452064878693868],12.452065,129.101563
3,6.087297,351.14029,14.854145,-409.697237,54.810996,[-120.92757985478572],[-10.779313355648197],10.779313,120.927580
4,6.071949,349.247901,14.854145,-407.267357,52.249574,[-117.37853370165767],[-9.824228921718841],9.824229,117.378534
5,6.098117,352.478003,14.854145,-411.416337,56.683601,[-114.00268270341391],[-9.809326951159148],9.809327,114.002683
6,6.097531,352.40551,14.854145,-411.323146,56.580783,[-112.40523916526774],[-9.524102416389553],9.524102,112.405239
7,6.089374,351.396879,14.854145,-410.026889,55.16616,[-110.74393933304167],[-9.081237570833316],9.081238,110.743939
8,6.082127,350.502251,14.854145,-408.877711,53.936032,[-109.48964450224787],[-8.737978638001909],8.737979,109.489645
9,6.078363,350.038091,14.854145,-408.281693,53.306802,[-108.67378395665627],[-8.538401313132436],8.538401,108.673784


In [23]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
